<center><h2>Sale Prices Prediction for Iowa Residential Houses</h2></center>
<center>(ongoing)</center>

<center>Version 1: Initial modeling with Decision Tree and Random Forest</center>

#### 1. Import modules and datasets

In [193]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [194]:
train_file_path = "~/PROJECTS/05_iowa_house_sale_price_prediction/train.csv"
train_full = pd.read_csv(train_file_path, index_col = "Id")

In [195]:
train_full.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [196]:
test_file_path = "~/PROJECTS/05_iowa_house_sale_price_prediction/test.csv"
test_full = pd.read_csv(test_file_path, index_col = "Id")

In [197]:
test_full.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


#### 2. Specify the target and the features

In [198]:
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = train_full[features]

In [199]:
y = train_full["SalePrice"]

In [200]:
X_test = X.copy()

#### 3. Split data into training data and validation data

In [201]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1)

#### 4. Decision Tree model

In [202]:
# create model

dtree_model = DecisionTreeRegressor(random_state=1)

In [203]:
# fit model

dtree_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=1)

In [204]:
# use validation data to predict

dtree_predictions = dtree_model.predict(X_valid)

In [205]:
# evaluate model with MAE

dtree_mae = mean_absolute_error(dtree_predictions, y_valid)
print("Validation MAE when not specifying max_leaf_nodes: {:,.2f}".format(dtree_mae))

Validation MAE when not specifying max_leaf_nodes: 29,652.93


In [206]:
# create function to compare MAE with different max_leaf_nodes arguments

def get_mae(max_leaf_nodes, X_train, X_valid, y_train, y_valid):
    """
    This function is used to compare the Mean Absolute Error scores from different values 
    for the max_leaf_nodes argument to avoid underfitting or overfitting.
    """
    model = DecisionTreeRegressor(max_leaf_nodes = max_leaf_nodes, random_state=0)
    model.fit(X_train, y_train)
    preds_valid = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, preds_valid)
    return mae

In [207]:
# get the best value for max_leaf_nodes

for max_leaf_nodes in [5, 25, 50, 100, 250, 500]:
    my_mae = get_mae(max_leaf_nodes, X_train, X_valid, y_train, y_valid)
    print("Max leaf nodes: %d \t Mean Absolute Error: %d"%(max_leaf_nodes, my_mae))

Max leaf nodes: 5 	 Mean Absolute Error: 35044
Max leaf nodes: 25 	 Mean Absolute Error: 29016
Max leaf nodes: 50 	 Mean Absolute Error: 27405
Max leaf nodes: 100 	 Mean Absolute Error: 27282
Max leaf nodes: 250 	 Mean Absolute Error: 27893
Max leaf nodes: 500 	 Mean Absolute Error: 29454


In [208]:
# use best value for max_leaf_nodes to remodeling

dtree_model = DecisionTreeRegressor(max_leaf_nodes=100, random_state=1)

dtree_model.fit(X_train, y_train)

dtree_predictions = dtree_model.predict(X_valid)

dtree_mae = mean_absolute_error(dtree_predictions, y_valid)
print("Validation MAE for best value of max_leaf_nodes: {:,.2f}".format(dtree_mae_2))

Validation MAE for best value of max_leaf_nodes: 27,282.51


In [209]:
# generate predictions with test data

dtree_predictions_test = dtree_model.predict(X_test)

In [210]:
# save result

dtree_result = pd.DataFrame({"Id": X_test.index, 
                             "SalePrice": dtree_predictions_test})

dtree_result.head()

,Id,SalePrice
0,1,205665.277778
1,2,148515.311111
2,3,234387.032258
3,4,127746.153846
4,5,290534.894737


#### 5. Random Forest model

In [211]:
# create model

rforest_model = RandomForestRegressor(random_state = 0)

In [212]:
# fit model

rforest_model.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [213]:
# use validation data to predict

rforest_predictions = rforest_model.predict(X_valid)

In [220]:
# evaluate model with MAE

rforest_mae = mean_absolute_error(rforest_predictions, y_valid)
print("Validation MAE with no specifying inputs: {:,.2f}".format(rforest_mae))

Validation MAE with no specifying inputs: 22,118.63


In [221]:
# create function to compare MAE with different Random Forest models

def score_model(model, X_t = X_train, X_v = X_valid, y_t = y_train, y_v = y_valid):
    """
    This function is used to compare the Mean Absolute Error scores from different inputs 
    in modeling to avoid underfitting or overfitting.
    """
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

In [222]:
# Define models with different inputs

model_1 = RandomForestRegressor(n_estimators=50, random_state=1)
model_2 = RandomForestRegressor(n_estimators=100, random_state=1)
model_3 = RandomForestRegressor(n_estimators=100, criterion='mae', random_state=1)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=1)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=1)

models = [model_1, model_2, model_3, model_4, model_5]

In [223]:
# get the best model inputs

for i in range(len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))

Model 1 MAE: 22024
Model 2 MAE: 21857
Model 3 MAE: 21878
Model 4 MAE: 22669
Model 5 MAE: 22352


In [224]:
# use best inputs to remodeling

rforest_model = RandomForestRegressor(n_estimators=100, random_state=1)

rforest_model.fit(X_train, y_train)

rforest_predictions = rforest_model.predict(X_valid)

rforest_mae = mean_absolute_error(rforest_predictions, y_valid)
print("Validation MAE for best inputs: {:,.2f}".format(rforest_mae))

Validation MAE for best inputs: 21,857.16


In [225]:
# generate predictions with test data

rforest_predictions_test = rforest_model.predict(X_test)

In [229]:
# save result

rforest_result = pd.DataFrame({"Id": X_test.index, 
                               "SalePrice": rforest_predictions_test})

rforest_result.head()

,Id,SalePrice
0,1,210037.10
1,2,175173.00
2,3,220261.78
3,4,138555.74
4,5,261146.56
